In [1]:
%matplotlib inline

递归神经网络 - Recurrent Neural Network
====
>Python2.7 + Pytorch 1.2.0 backened
>
>text

本节主要参考：
http://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html#sphx-glr-beginner-nlp-word-embeddings-tutorial-py

In [2]:
# -*- coding: utf-8 -*-
# !/usr/bin/env python

from __future__ import print_function

In [3]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F

torch.manual_seed(1)

1.N-Gram Language Modeling <br>
probability{ x_(i-N),x_(i-N+1),x_(i-N+2),...,x_(i-1) -> x_(i) }

based on word embedding

In [4]:
# hyper-parameter
training_epoch = 10000
learning_rate = 1e-3
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10

In [5]:
# We will use Shakespeare Sonnet 2
test_sentence = """
Deep learning (also known as deep structured learning or hierarchical learning)
is part of a broader family of machine learning methods based on learning data
representations, as opposed to task-specific algorithms. Learning can be supervised,
semi-supervised or unsupervised. Deep learning models are loosely related to information
processing and communication patterns in a biological nervous system, such as neural
coding that attempts to define a relationship between various stimuli and associated
neuronal responses in the brain. Deep learning architectures such as deep neural
networks, deep belief networks and recurrent neural networks have been applied to
fields including computer vision, speech recognition, natural language processing,
audio recognition, social network filtering, machine translation, bioinformatics
and drug design, where they have produced results comparable to and in some
cases superior to human experts.
""".split()

vocab = set(test_sentence)
word2ind = {word: i for i, word in enumerate(vocab)}
ind2word = {i: word for i, word in enumerate(vocab)}

data_num = len(test_sentence) - CONTEXT_SIZE
x = [[word2ind[test_sentence[i]], word2ind[test_sentence[i + 1]]]
          for i in range(len(test_sentence) - CONTEXT_SIZE)]
y = [[word2ind[test_sentence[i]]] for i in range(CONTEXT_SIZE, len(test_sentence))]

In [6]:
class NGramLanguageModeler(nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)

        self.hidden_size = 32
        self.num_layers = 2
        self.BiLSTM = nn.LSTM(input_size=embedding_dim, hidden_size=self.hidden_size,
                              num_layers=self.num_layers, batch_first=True, bidirectional=True)

        self.fc = nn.Linear(context_size * self.hidden_size * 2, vocab_size)

    def forward(self, x):
        h_0 = Variable(torch.zeros(2 * self.num_layers, x.size(0), self.hidden_size)).cuda()
        c_0 = Variable(torch.zeros(2 * self.num_layers, x.size(0), self.hidden_size)).cuda()

        embeds = self.embeddings(x)
        out, _ = self.BiLSTM(embeds, (h_0, c_0))
        out = out.contiguous().view(embeds.size(0), -1)
        out = self.fc(out)
        return out


model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
model.cuda()
print(model)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adagrad(model.parameters(), lr=learning_rate)

NGramLanguageModeler (
  (embeddings): Embedding(95, 10)
  (BiLSTM): LSTM(10, 32, num_layers=2, batch_first=True, bidirectional=True)
  (fc): Linear (128 -> 95)
)


In [7]:
x = Variable(torch.LongTensor(x)).cuda()
y = Variable(torch.LongTensor(y)).cuda()
for epoch in range(1, 1 + training_epoch):
    model.zero_grad()
    y_ = model(x)
    loss = criterion(y_, y.view(-1))
    loss.backward()
    optimizer.step()

    # 因为cost value不太明显，显示cost之和
    if epoch % 1000 == 0:
        print('Epoch %s / %s, training cost: %s' % (epoch, training_epoch, loss.data[0]))

Epoch 1000 / 10000, training cost: 3.00195384026
Epoch 2000 / 10000, training cost: 1.64184463024
Epoch 3000 / 10000, training cost: 0.890686273575
Epoch 4000 / 10000, training cost: 0.531099021435
Epoch 5000 / 10000, training cost: 0.353038758039
Epoch 6000 / 10000, training cost: 0.258097320795
Epoch 7000 / 10000, training cost: 0.202841848135
Epoch 8000 / 10000, training cost: 0.168025612831
Epoch 9000 / 10000, training cost: 0.144640639424
Epoch 10000 / 10000, training cost: 0.128112345934


In [8]:
# generating text
model.eval()
context_idxs = [word2ind['Deep'], word2ind['Learning']]
logue = context_idxs
for i in xrange(data_num):
    context_var = Variable(torch.LongTensor(context_idxs).view(1, -1)).cuda()
    context_idxs = model(context_var).data.topk(1)[1].cpu().numpy()[0, 0]
    logue.append(context_idxs)
    context_idxs = logue[-2:]

pred_sentence = ' '.join([ind2word[i] for i in logue])

In [9]:
import editdistance

print('Distance between these two sentences is %s' % (editdistance.eval(' '.join(test_sentence), pred_sentence)))
print("\033[1;31;40m %s \033[0m" % (' '.join(test_sentence)))
print(pred_sentence)

Distance between these two sentences is 573
 Deep learning (also known as deep structured learning or hierarchical learning) is part of a broader family of machine learning methods based on learning data representations, as opposed to task-specific algorithms. Learning can be supervised, semi-supervised or unsupervised. Deep learning models are loosely related to information processing and communication patterns in a biological nervous system, such as neural coding that attempts to define a relationship between various stimuli and associated neuronal responses in the brain. Deep learning architectures such as deep neural networks, deep belief networks and recurrent neural networks have been applied to fields including computer vision, speech recognition, natural language processing, audio recognition, social network filtering, machine translation, bioinformatics and drug design, where they have produced results comparable to and in some cases superior to human experts. 
Deep Learning (